In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

RDD 만을 이용한 데이터 추출

In [3]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [19]:
movie_att = movies_rdd.join(attendances_rdd)
movie_att = movie_att.sortBy(lambda x: x[0])
movie_att.take(10)

[(1, (('어벤져스', '마블'), (13934592, 'KR'))),
 (2, (('슈퍼맨', 'DC'), (2182227, 'KR'))),
 (3, (('배트맨', 'DC'), (4226242, 'KR'))),
 (4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (5, (('아이언맨', '마블'), (4300365, 'KR')))]

In [20]:
movie_att.filter(
    lambda x: x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [21]:
sc.stop()

In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [23]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

### 스키마를 알아야 한다.

In [26]:
movie_schema = ["id", "name", "company", "year", "month", "day"]

### 2. 데이터 프레임 만들기

In [31]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [32]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [33]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [54]:
# 2013 이후 영화
df.filter(df.year > 2013).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [63]:
# 마블 or DC
df.filter((df.company == "마블") | (df.company == "DC")).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [34]:
df.createOrReplaceTempView("movies")

### 영화 이름만 가져오기

In [39]:
query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



### 2010년 이후에 개봉한 영화를 조회

In [40]:
query = """

SELECT *
FROM movies
WHERE year >= 2010

"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



### 2012년도 이전에 개봉한 영화의 이름과 회사를 출력

In [42]:
query = """

SELECT name, company
FROM movies
WHERE year < 2012

"""
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|  배트맨|     DC|
|아이언맨|   마블|
+--------+-------+



### like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때

### % 기호를 사용해서 문장이 매칭되는지 확인 가!
### 제목이 ~맨으로 끝나는 데이터의 모든 정보를 조회

In [43]:
query = """

SELECT *
FROM movies
WHERE name like "%맨"

"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



### BETWEEN 특정 데이터와 데이터 사이를 조회

### 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8

In [45]:
query = """

SELECT *
FROM movies
WHERE month BETWEEN 4 and 8

"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



### Join 구현하기

In [46]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [47]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [50]:
# 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
# StructField : 컬럼
att_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [51]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [52]:
att_df.createOrReplaceTempView("att")